In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [4]:
# Load waste data from CSV export
csv_path = Path("reports/waste_log_export.csv")
df = pd.read_csv(csv_path)
df['notes'] = df['notes'].fillna('')
print(f"Loaded {len(df)} entries")
df.head()

Loaded 34 entries


,id,timestamp,station,waste_type,item_name,quantity_type,quantity_value,notes
0,c16bc56a-0317-43cc-8a0d-46da943efeec,2025-12-29T15:36:37.430691,middle,trim,pork rack,lbs,3.75,
1,29b6f813-d3ec-417e-8280-90c0946cb69c,2025-12-29T16:22:32.761043,grill,trim,new york,lbs,4.50,dry aged
2,7982dec0-6236-487f-801a-aa2c7518f309,2025-12-31T13:07:36.190794,grill,trim,Filet,lbs,0.75,
3,c58fdce9-7bc8-429a-9acc-e3c429b6372e,2025-12-31T13:09:04.851538,grill,trim,Ribeye,lbs,2.12,
4,29ea9d5a-ea6e-4059-9f51-4ce3223703c7,2025-12-31T13:10:16.927261,other_prep,spoilage,red pepper,lbs,3.00,
